In [1]:
import os
import json
import pandas as pd
import torch
from transformers import AutoTokenizer

from torchmetrics.functional import accuracy, f1_score, auroc
from torchsummary import summary

from classes.TwitterDataset import TwitterDataset
from classes.Heirarchichal import HeirarchichalNN
from classes.MultiTask import MultiTaskNN
from classes.SingleTask import SingleTaskNN


C:\Users\Arian\.conda\envs\ai-ds-test4\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.set_printoptions(sci_mode=False)

In [3]:

with open('./config/config.json', 'r') as f:
    config = json.load(f)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



In [4]:
test_df = pd.read_csv('../res/preprocessed/test_final.csv')

tokenizer = AutoTokenizer.from_pretrained(config['bert_model_name'])
test_dataset = TwitterDataset(
    test_df, tokenizer, max_token_len=config['max_token_len'])

LABEL_COLUMNS = list(test_df.columns)
LABEL_COLUMNS.remove('text')

TASK1_LABELS = LABEL_COLUMNS[:3]
TASK2_LABELS = LABEL_COLUMNS[3:4]
TASK3_LABELS = LABEL_COLUMNS[4:]


task1_id2label = {idx: label for idx, label in enumerate(TASK1_LABELS)}
task1_label2id = {label: idx for idx, label in enumerate(TASK1_LABELS)}

task2_label2id = {label: idx for idx, label in enumerate(TASK2_LABELS)}
task2_id2label = {idx: label for idx, label in enumerate(TASK2_LABELS)}

task3_label2id = {label: idx for idx, label in enumerate(TASK3_LABELS)}

task3_id2label = {idx: label for idx, label in enumerate(TASK3_LABELS)}


In [13]:
config["heirarchichal_model"]

'heirarchichal_20.pt'

### LOAD Models

In [6]:
print('---------LOADING MODEL---------')
PATH = './torch_model'

model_names_in_config = ["heirarchichal_model",
                         "multitask_model", "singletask_model_1", "singletask_model_2","singletask_model_3"]
loaded_models = []
singletask_taskn = {"task1":3,"task2":1,"task3":5}
task_names = list(singletask_taskn)
task_n_classes = list(singletask_taskn.values())

for idx, model_name in enumerate(model_names_in_config):    
    if model_name == "heirarchichal_model":    
        loaded_model = HeirarchichalNN(bert_model_name=config['bert_model_name'])
    elif model_name == "multitask_model":
        loaded_model = MultiTaskNN(bert_model_name=config['bert_model_name'])
    else:
        loaded_model = SingleTaskNN(task_name=task_names[idx-2], taskn_classes=task_n_classes[idx-2],
            bert_model_name=config['bert_model_name'])
    print(model_name)
    loaded_model.load_state_dict(torch.load(
    os.path.join(PATH, config[model_name]), map_location=device))

    loaded_model = loaded_model.to(device)
    loaded_model.eval()
    loaded_model.freeze()

    loaded_models.append(loaded_model)
    loaded_model = None


---------LOADING MODEL---------


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


heirarchichal_model


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


multitask_model


...

singletask_model_1


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


singletask_model_2


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


singletask_model_3


In [7]:
def get_accuracy_score_multi_output(model):
    task1_predictions = []
    task1_labels = []

    task2_predictions = []
    task2_labels = []

    task3_predictions = []
    task3_labels = []

    with torch.no_grad():
        for idx, item in enumerate(test_dataset):
            
            _, prediction = model(
                item['input_ids'].unsqueeze(dim=0).to(device),
                item['attention_mask'].unsqueeze(dim=0).to(device)
            )

            task1_predictions.append(prediction[0].flatten())
            task2_predictions.append(prediction[1].flatten())
            task3_predictions.append(prediction[2].flatten())

            task1_labels.append(item['labels1'].int())
            task2_labels.append(item['labels2'].int())
            task3_labels.append(item['labels3'].int())

    task1_predictions_ = torch.stack(task1_predictions).detach().cpu()
    task1_labels_ = torch.stack(task1_labels).detach().cpu()
   #  print(task1_predictions)
   #  print(task2_predictions)
    task2_predictions_ = torch.stack(task2_predictions).detach().cpu()
    task2_labels_ = torch.stack(task2_labels).detach().cpu()

    task3_predictions_ = torch.stack(task3_predictions).detach().cpu()
    task3_labels_ = torch.stack(task3_labels).detach().cpu()

    THRESHOLD = 0.5

    task1_accuracy_score = accuracy(
        task1_predictions_, task1_labels_, threshold=THRESHOLD)

    task2_accuracy_score = accuracy(
        task2_predictions_, task2_labels_, threshold=THRESHOLD)

    task3_accuracy_score = accuracy(
        task3_predictions_, task3_labels_, threshold=THRESHOLD)

    print(f'task1_accuracy_score: {task1_accuracy_score}')
    print(f'task2_accuracy_score: {task2_accuracy_score}')
    print(f'task3_accuracy_score: {task3_accuracy_score}')

In [6]:
def get_accuracy_score_single_output(model, task_idx):
    task_predictions = []
    task_labels = []
    label = 'labels'+str(task_idx-1)
    with torch.no_grad():
        for idx, item in enumerate(test_dataset):
            
            _, prediction = model(
                item['input_ids'].unsqueeze(dim=0).to(device),
                item['attention_mask'].unsqueeze(dim=0).to(device)
            )

            task_predictions.append(prediction.flatten())
            task_labels.append(item[label].int())

    task_predictions_ = torch.stack(task_predictions).detach().cpu()
    task_labels_ = torch.stack(task_labels).detach().cpu()
    THRESHOLD = 0.5

    task_accuracy_score = accuracy(
        task_predictions_, task_labels_, threshold=THRESHOLD)

    print(f'task{task_idx}_accuracy_score: {task_accuracy_score}')

In [7]:
for idx, model in enumerate(loaded_models):
   print(model_names_in_config[idx])
   if idx > 1:
      get_accuracy_score_single_output(model, task_idx=idx)
   else:
      get_accuracy_score_multi_output(model)
      


NameError: name 'loaded_models' is not defined

In [10]:
device = ('cpu')
PATH = './torch_model'
loaded_models = []
singletask_taskn = {"task1": 3, "task2": 1, "task3": 5}
task_names = list(singletask_taskn)
task_n_classes = list(singletask_taskn.values())

for i in range(3):
   label = f'singletask{i+1}.pt'
   loaded_model = SingleTaskNN(task_name=task_names[i], taskn_classes=task_n_classes[i],
                               bert_model_name=config['bert_model_name'])
   loaded_model.load_state_dict(torch.load(
    os.path.join(PATH, label), map_location=device))

   loaded_model = loaded_model.to(device)
   loaded_model.eval()
   loaded_model.freeze()

   loaded_models.append(loaded_model)
   loaded_model = None


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RuntimeError: Error(s) in loading state_dict for SingleTaskNN:
	Missing key(s) in state_dict: "bert.embeddings.position_ids", "bert.embeddings.token_type_embeddings.weight", "bert.encoder.layer.0.attention.self.query.weight", "bert.encoder.layer.0.attention.self.query.bias", "bert.encoder.layer.0.attention.self.key.weight", "bert.encoder.layer.0.attention.self.key.bias", "bert.encoder.layer.0.attention.self.value.weight", "bert.encoder.layer.0.attention.self.value.bias", "bert.encoder.layer.0.attention.output.dense.weight", "bert.encoder.layer.0.attention.output.dense.bias", "bert.encoder.layer.0.attention.output.LayerNorm.weight", "bert.encoder.layer.0.attention.output.LayerNorm.bias", "bert.encoder.layer.0.intermediate.dense.weight", "bert.encoder.layer.0.intermediate.dense.bias", "bert.encoder.layer.0.output.dense.weight", "bert.encoder.layer.0.output.dense.bias", "bert.encoder.layer.0.output.LayerNorm.weight", "bert.encoder.layer.0.output.LayerNorm.bias", "bert.encoder.layer.1.atte

In [5]:
get_accuracy_score_single_output(loaded_models[2], 2)

NameError: name 'get_accuracy_score_single_output' is not defined

In [ ]:
# if idx%500 == 0:
#     print(test_df.iloc[idx].text)
#     print('TASK 1')
#     print(prediction[0])
#     print(task1_id2label)
#     print(task1_id2label[int(torch.argmax(prediction[0]))])

#     print('TASK 2')
#     print(prediction[1])
#     print(task2_id2label)
#     print(task2_id2label[int(torch.argmax(prediction[1]))])

#     print('TASK 3')
#     print(prediction[2])
#     print(task3_id2label)
#     print(task3_id2label[int(torch.argmax(prediction[2]))])
#     print('=============================')
